In [1]:
import pandas as pd
import json
from unidecode import unidecode
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

In [13]:
with open('data.json', encoding="utf8") as json_file:
    data = json.load(json_file)

df = pd.DataFrame.from_dict(data)
df.columns = ['token', 'attributes', 'description']
df = pd.concat([df.drop(['attributes'], axis=1), df['attributes'].apply(pd.Series)], axis=1).drop_duplicates()
df.head()

,token,description,کارکرد,مدل (سال تولید),رنگ,برند و تیپ,وضعیت موتور,وضعیت شاسی‌ها,وضعیت بدنه,مهلت بیمهٔ شخص ثالث,گیربکس,نحوه فروش,قیمت,فروشنده,وضعیت شاسی جلو,وضعیت شاسی عقب,نمایشگاه
0,gYrWgJZ1,سالم سالم خشک فقط ۲۰۰۰ تا کار کرده,۲٬۰۰۰,۱۴۰۰,سفید,شاهین، G,سالم,سالم و پلمپ,سالم و بی‌خط و خش,۱۲ ماه,دنده‌ای,نقدی,۳۶۸٬۰۰۰٬۰۰۰ تومان,NaN,NaN,NaN,NaN
1,gYrWALmA,کد آگهی: 152170\n\nاین خودرو توسط کارنامه کارش...,۱۱۶٬۴۸۰,۱۳۹۶,سفید,رنو، تندر 90، E2 بنزینی,سالم,سالم و پلمپ,رنگ‌شدگی در ۱ ناحیه,۹ ماه,دنده‌ای,نقدی,۳۱۲٬۰۰۰٬۰۰۰ تومان,کارنامه,NaN,NaN,NaN
2,gYrWgEn9,با سلام \nتمام ادوات برقی سالم می‌باشد \nتمام ...,۲۵۰٬۰۰۰,۱۳۸۴,نقره‌ای,مزدا، 323، دنده‌ای,سالم,سالم و پلمپ,رنگ‌شدگی در ۲ ناحیه,۶ ماه,دنده‌ای,نقدی,۲۵۰٬۰۰۰٬۰۰۰ تومان,NaN,NaN,NaN,NaN
3,gYrOg3vX,خودرو سالم و بیرنگ \nکولر بخاری روشن\nکم کار ...,۱۰۰٬۰۰۰,۱۳۸۵,یشمی,پژو، RDI، بنزینی,سالم,سالم و پلمپ,رنگ‌شدگی در ۱ ناحیه,۷ ماه,دنده‌ای,نقدی,۶۸٬۰۰۰٬۰۰۰ تومان,NaN,NaN,NaN,NaN
4,gYrSQ0aN,تیپ دو سفارش مدیران مدل ۸۸ _ فنی سلامت _ لوازم...,۲۵۰٬۰۰۰,۱۳۸۸,خاکستری,پژو، 206، تیپ ۲,سالم,سالم و پلمپ,خط و خش جزیی,۱۱ ماه,دنده‌ای,نقدی,۱۴۲٬۰۰۰٬۰۰۰ تومان,NaN,NaN,NaN,NaN


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4777 entries, 0 to 10879
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   token                4777 non-null   object
 1   description          4777 non-null   object
 2   کارکرد               4777 non-null   object
 3   مدل (سال تولید)      4777 non-null   object
 4   رنگ                  4777 non-null   object
 5   برند و تیپ           4777 non-null   object
 6   وضعیت موتور          4683 non-null   object
 7   وضعیت شاسی‌ها        4289 non-null   object
 8   وضعیت بدنه           4749 non-null   object
 9   مهلت بیمهٔ شخص ثالث  4579 non-null   object
 10  گیربکس               4581 non-null   object
 11  نحوه فروش            4534 non-null   object
 12  قیمت                 4777 non-null   object
 13  فروشنده              131 non-null    object
 14  وضعیت شاسی جلو       276 non-null    object
 15  وضعیت شاسی عقب       266 non-null    object
 16  نمایش

In [15]:
df['token'].nunique()

4414

In [12]:
df.iloc[:,-2].unique()

array([nan, 'ضربه\u200cخورده', 'سالم و پلمپ', 'رنگ\u200cشده'],
      dtype=object)

In [9]:
df = df[df['مدل (سال تولید)'] != 'قبل از ۱۳۶۶']

In [10]:
df['کارکرد'] = df['کارکرد'].apply(lambda x: unidecode(x)).str.replace(',', '').astype(int)

ValueError: invalid literal for int() with base 10: 'khrkhrd'

In [11]:
df['مدل (سال تولید)'] = df['مدل (سال تولید)'].apply(lambda x: unidecode(x)).str.replace(',', '').astype(int)

ValueError: invalid literal for int() with base 10: 'mdl (sl twlyd)'

In [ ]:
df['مهلت بیمهٔ شخص ثالث'] = df['مهلت بیمهٔ شخص ثالث'].str.replace('ماه', '').apply(lambda x: unidecode(str(x))).astype(float)

In [ ]:
df['قیمت'] = df['قیمت'].str.replace('تومان', '').apply(lambda x: unidecode(x)).str.replace(',', '').str.extract('(\d+)').astype(float)

In [ ]:
df.head()

In [ ]:
data = df.copy().drop(['token', 'description', 'نمایشگاه', 'فروشنده', 'وضعیت شاسی عقب', 'وضعیت شاسی جلو', 'برند و تیپ'], axis = 1)

In [ ]:
categorical_cols = []
for col in data.columns:
    if data[col].dtypes == 'object':
        categorical_cols.append(col)
print(categorical_cols)

In [ ]:
data = pd.concat([data, pd.get_dummies(data[categorical_cols], drop_first=True)], axis=1)
data.drop(categorical_cols, axis=1, inplace=True)
data.head()

In [ ]:
for col in categorical_cols:
    print(col, df[col].nunique())

In [ ]:
X = data.dropna().iloc[:, data.columns!='قیمت']
y = data.dropna().iloc[:, 3]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=4)

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
regressor_lr = LinearRegression()
regressor_lr.fit(X_train, y_train)

In [ ]:
regressor_lr.score(X_train, y_train)

In [ ]:
regressor_lr.score(X_test, y_test)